<a href="https://colab.research.google.com/github/biranchi2018/simpletransformer-examples/blob/master/Multilabel_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install simpletransformers

In [3]:
import pandas as pd
from simpletransformers.classification import MultiLabelClassificationModel

# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns, a 'text' and a 'labels' column. The `labels` column should contain multi-hot encoded lists.
train_data = [["Example sentence 1 for multilabel classification.", [1, 1, 1, 1, 0, 1]]] + [
    ["This is another example sentence. ", [0, 1, 1, 0, 0, 0]]
]
train_df = pd.DataFrame(train_data, columns=["text", "labels"])

eval_data = [
    ["Example eval sentence for multilabel classification.", [1, 1, 1, 1, 0, 1]],
    ["Example eval senntence belonging to class 2", [0, 1, 1, 0, 0, 0]],
]
eval_df = pd.DataFrame(eval_data)

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    use_cuda=True,
    num_labels=6,
    args={'fp16':False, "reprocess_input_data": True, "overwrite_output_dir": True, "num_train_epochs": 5},
)
# You can set class weights by using the optional weight argument
print(train_df.head())

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
print(model_outputs)

predictions, raw_outputs = model.predict(["This thing is entirely different from the other thing. "])
print(predictions)
print(raw_outputs)


                                                text              labels
0  Example sentence 1 for multilabel classification.  [1, 1, 1, 1, 0, 1]
1                 This is another example sentence.   [0, 1, 1, 0, 0, 0]


Running loss: 0.751533


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.715369


Running loss: 0.711920


Running loss: 0.698078


Running loss: 0.697137



/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'LRAP': 0.6916666666666667, 'eval_loss': 0.6917680501937866}
[[0.49289387 0.52271587 0.48739725 0.54709655 0.48671624 0.48618382]
 [0.49372256 0.50634307 0.48073784 0.547069   0.4990852  0.48219553]]



[[0, 1, 0, 1, 0, 0]]
[[0.48949927 0.5156814  0.48203737 0.54251534 0.4846027  0.47872642]]


## Toxic Comments Classification


https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/

https://towardsdatascience.com/multi-label-classification-using-bert-roberta-xlnet-xlm-and-distilbert-with-simple-transformers-b3e0cda12ce5


In [0]:
import pandas as pd
import os, shutil
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel


In [0]:
# if os.path.isdir("outputs")==True:
#   shutil.rmtree("outputs")

In [19]:
df = pd.read_csv('data/train.csv', engine='python', error_bad_lines=False)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [20]:
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))
df['text'] = df['comment_text'].apply(lambda x: x.replace('\n', ' '))

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",Explanation Why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",D'aww! He matches this background colour I'm s...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","Hey man, I'm really not trying to edit war. It..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",""" More I can't make any real suggestions on im..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","You, sir, are my hero. Any chance you remember..."


In [0]:
train_df, eval_df = train_test_split(df, test_size=0.2)

In [0]:
model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=6, use_cuda=True,  args={'fp16':False, 'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512, 'overwrite_output_dir': True,})
model.train_model(train_df)

Running loss: 0.652043

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.015481

In [0]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [0]:
result

In [0]:
model_outputs

In [0]:
wrong_predictions

In [0]:
test_df = pd.read_csv('data/test.csv', engine='python', error_bad_lines=False)
test_df.head()

In [0]:
to_predict = test_df.comment_text.apply(lambda x: x.replace('\n', ' ')).tolist()
preds, outputs = model.predict(to_predict)

In [0]:
preds

In [0]:
outputs

In [0]:
sub_df = pd.DataFrame(outputs, columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])


In [0]:
sub_df['id'] = test_df['id']
sub_df = sub_df[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]

sub_df.to_csv('outputs/submission.csv', index=False)